In [28]:
import ollama
import ast
import json

In [21]:
with open('../Outputs/Preference/low_budget+museum_1.txt', 'r') as f:
    text = f.read()

In [15]:
prompt = 'You are a helper robot help with evaluating the travel plan. Your job is to extract all the business mentioned in the time schedule, including the name of the restaurants, hotels, and attractions. Only keep the information that is in the time slot, not general recommendations at the bottom. Parse the information into array of string format. You should only output this array of strings, nothing else! Here is the travel plan: ' + text
response = ollama.generate(model='llama3.1', prompt=prompt, options={'num_ctx': 10000})
with open("../Evaluation/Helper/low_budget+museum_1.txt", "w") as f:
    f.write(response['response'])

In [ ]:
#cast a string into an array
array = ast.literal_eval(response['response'])
array

["Pat's King of Steaks",
 'Philadelphia Museum of Art',
 'Franklin Institute Science Museum',
 'Schuylkill River Trail',
 'El Vez',
 'Aloft Philadelphia Airport',
 'Green Eggs Cafe',
 'Independence Hall',
 'Reading Terminal Market',
 'Museum of American Jewish History']

In [24]:
array[3]

'Schuylkill River Trail'

In [52]:
all = []

#load all_pruned.jsonl into a list of dictionaries
with open('../Datasets/all_pruned.jsonl', 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append to the list
        json_obj = json.loads(line.strip())  # strip() removes any leading/trailing whitespace
        all.append(json_obj)

In [53]:
mentioned = []

for business in array:
    for place in all:
        if place['name'] == business:
            mentioned.append(place)
len(mentioned)

8

In [54]:
for mention in mentioned:
    print(mention['name'])

Pat's King of Steaks
Philadelphia Museum of Art
Schuylkill River Trail
El Vez
Aloft Philadelphia Airport
Green Eggs Cafe
Independence Hall
Reading Terminal Market


In [55]:
array

["Pat's King of Steaks",
 'Philadelphia Museum of Art',
 'Franklin Institute Science Museum',
 'Schuylkill River Trail',
 'El Vez',
 'Aloft Philadelphia Airport',
 'Green Eggs Cafe',
 'Independence Hall',
 'Reading Terminal Market',
 'Museum of American Jewish History']

In [ ]:
# dataset vs output
# National Museum of American Jewish History vs Museum of American Jewish History
# The Franklin Institute Science Museum vs Franklin Institute Science Museum
# need to use similarity score here haha.